In [16]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

In [2]:
csv_file = "output_data/cities.csv"
city_data = pd.read_csv(csv_file)
city_data

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,cherskiy,68.7500,161.3000,28.53,89,100,11.25,RU,1619671030
1,1,kaitangata,-46.2817,169.8464,59.00,67,89,8.99,NZ,1619671031
2,2,lebu,-37.6167,-73.6500,53.82,90,77,3.13,CL,1619671031
3,3,clyde river,70.4692,-68.5914,23.00,100,90,6.91,CA,1619671031
4,4,havre-saint-pierre,50.2334,-63.5986,39.20,93,75,5.75,CA,1619671032
...,...,...,...,...,...,...,...,...,...,...
549,549,dunshaughlin,53.5125,-6.5400,35.60,93,40,3.44,IE,1619671221
550,550,hirara,24.8000,125.2833,73.40,78,75,11.50,JP,1619671153
551,551,hun,29.1268,15.9477,85.14,10,95,7.47,LY,1619671221
552,552,waslala,13.2333,-85.3833,68.50,83,99,0.81,NI,1619671222


In [3]:
gmaps.configure(api_key=g_key)

In [4]:
locations = city_data[["Lat", "Lng"]].astype(float)
humidity  = city_data["Humidity"].astype(float)


In [5]:
fig = gmaps.figure(layout={
        'width': '800px',
        'height': '600px',
        'padding': '3px',
        'border': '1px solid black'
}, zoom_level=2, center=(46.0, -5.0))

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 2)

fig.add_layer(heat_layer)

fig


Figure(layout=FigureLayout(border='1px solid black', height='600px', padding='3px', width='800px'))

In [7]:
filtered_city_data = city_data.loc[(city_data['Max Temp']<80) & 
                                   (city_data['Max Temp']>70) & 
                                   (city_data['Wind Speed']<10) & 
                                   (city_data['Cloudiness']==0),:]
filtered_city_data

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
108,108,mildura,-34.2000,142.1500,77.00,25,0,5.75,AU,1619671067
133,133,esik,43.3552,77.4525,73.40,33,0,6.71,KZ,1619671076
228,228,buraydah,26.3260,43.9750,73.40,53,0,4.61,SA,1619671108
304,304,aksu,41.1231,80.2644,78.80,9,0,1.36,CN,1619671135
488,488,shache,38.4167,77.2406,79.12,8,0,1.63,CN,1619671198
493,493,rio branco,-9.9747,-67.8100,73.40,100,0,3.44,BR,1619671199


In [8]:
len(filtered_city_data)

6

In [12]:
filtered_city_data = filtered_city_data.dropna()

In [33]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
hotels = []

for index, row in filtered_city_data.iterrows():
    lat = filtered_city_data['Lat'][index]
    lng = filtered_city_data['Lng'][index]
    params = {
        "location": f"{lat},{lng}",
        "keyword": "hotel",
        "radius": 5000,
        "type": "lodging",
        "key": g_key
    }
    try:
        response = requests.get(base_url, params=params).json()
        
        hotels.append(response['results'][0]['name'])    
    except IndexError:
        hotels.append("")
        pass
    

mildura
esik
buraydah
aksu
shache
rio branco


In [34]:
hotels


['Quality Hotel Mildura Grand',
 '',
 'فندق تاورز HOTELTOWERS',
 'Jianshe Hotel',
 'Xinsheng Hotel',
 'Gran Lumni Hotel']

In [35]:
hotel_df = filtered_city_data
hotel_df['Hotel Name'] = hotels
hotel_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
108,108,mildura,-34.2000,142.1500,77.00,25,0,5.75,AU,1619671067,Quality Hotel Mildura Grand
133,133,esik,43.3552,77.4525,73.40,33,0,6.71,KZ,1619671076,
228,228,buraydah,26.3260,43.9750,73.40,53,0,4.61,SA,1619671108,فندق تاورز HOTELTOWERS
304,304,aksu,41.1231,80.2644,78.80,9,0,1.36,CN,1619671135,Jianshe Hotel
488,488,shache,38.4167,77.2406,79.12,8,0,1.63,CN,1619671198,Xinsheng Hotel
493,493,rio branco,-9.9747,-67.8100,73.40,100,0,3.44,BR,1619671199,Gran Lumni Hotel


In [36]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [37]:
hotel_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(hotel_layer)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', padding='3px', width='800px'))